In [528]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

In [529]:
df = pd.read_csv(r"C:\Users\vishn\Downloads\hypothyroid.csv")
df=df.drop("TBG measured",axis=1)
df=df.drop("TBG",axis=1)
df=df.drop("referral source",axis=1)
df.head(10)


,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,binaryClass
0,41,F,f,f,f,f,f,f,f,f,...,1.3,t,2.5,t,125,t,1.14,t,109,P
1,23,F,f,f,f,f,f,f,f,f,...,4.1,t,2,t,102,f,?,f,?,P
2,46,M,f,f,f,f,f,f,f,f,...,0.98,f,?,t,109,t,0.91,t,120,P
3,70,F,t,f,f,f,f,f,f,f,...,0.16,t,1.9,t,175,f,?,f,?,P
4,70,F,f,f,f,f,f,f,f,f,...,0.72,t,1.2,t,61,t,0.87,t,70,P
5,18,F,t,f,f,f,f,f,f,f,...,0.03,f,?,t,183,t,1.3,t,141,P
6,59,F,f,f,f,f,f,f,f,f,...,?,f,?,t,72,t,0.92,t,78,P
7,80,F,f,f,f,f,f,f,f,f,...,2.2,t,0.6,t,80,t,0.7,t,115,P
8,66,F,f,f,f,f,f,f,f,f,...,0.6,t,2.2,t,123,t,0.93,t,132,P
9,68,M,f,f,f,f,f,f,f,f,...,2.4,t,1.6,t,83,t,0.89,t,93,P


In [530]:
print(df.describe())

         age   sex on thyroxine query on thyroxine on antithyroid medication  \
count   3772  3772         3772               3772                      3772   
unique    94     3            2                  2                         2   
top       59     F            f                  f                         f   
freq      95  2480         3308               3722                      3729   

        sick pregnant thyroid surgery I131 treatment query hypothyroid  ...  \
count   3772     3772            3772           3772              3772  ...   
unique     2        2               2              2                 2  ...   
top        f        f               f              f                 f  ...   
freq    3625     3719            3719           3713              3538  ...   

         TSH T3 measured    T3 TT4 measured   TT4 T4U measured   T4U  \
count   3772        3772  3772         3772  3772         3772  3772   
unique   288           2    70            2   242          

In [531]:
df.replace("?",np.nan,inplace=True)

df.sex.replace({'F':2,'M':1},inplace=True)
round_Values = round(df.sex.mean())
df.sex.fillna(round_Values,inplace=True)

df.sex.unique()
df = df.dropna(subset=["age"])

In [532]:
print(df.isnull().sum())

age                            0
sex                            0
on thyroxine                   0
query on thyroxine             0
on antithyroid medication      0
sick                           0
pregnant                       0
thyroid surgery                0
I131 treatment                 0
query hypothyroid              0
query hyperthyroid             0
lithium                        0
goitre                         0
tumor                          0
hypopituitary                  0
psych                          0
TSH measured                   0
TSH                          369
T3 measured                    0
T3                           769
TT4 measured                   0
TT4                          231
T4U measured                   0
T4U                          387
FTI measured                   0
FTI                          385
binaryClass                    0
dtype: int64


In [533]:
for label in df.columns:
    print(df[label].value_counts())
    print()
    print(df[label].unique())

age
59     95
60     91
70     90
73     81
55     81
       ..
10      1
4       1
5       1
455     1
6       1
Name: count, Length: 93, dtype: int64

['41' '23' '46' '70' '18' '59' '80' '66' '68' '84' '67' '71' '28' '65'
 '42' '63' '51' '81' '54' '55' '60' '25' '73' '34' '78' '37' '85' '26'
 '58' '64' '44' '48' '61' '35' '83' '21' '87' '53' '77' '27' '69' '74'
 '38' '76' '45' '36' '22' '43' '72' '82' '31' '39' '49' '62' '57' '1' '50'
 '30' '29' '75' '19' '7' '79' '17' '24' '15' '32' '47' '16' '52' '33' '13'
 '10' '89' '56' '20' '90' '40' '88' '14' '86' '94' '12' '4' '11' '8' '5'
 '455' '2' '91' '6' '93' '92']
sex
2.0    2629
1.0    1142
Name: count, dtype: int64

[2. 1.]
on thyroxine
f    3308
t     463
Name: count, dtype: int64

['f' 't']
query on thyroxine
f    3721
t      50
Name: count, dtype: int64

['f' 't']
on antithyroid medication
f    3728
t      43
Name: count, dtype: int64

['f' 't']
sick
f    3624
t     147
Name: count, dtype: int64

['f' 't']
pregnant
f    3718
t      

In [534]:
df["binaryClass"] =(df["binaryClass"] == "P").astype(int)
df["sex"] =(df["sex"]=="t").astype(int)
df["on thyroxine"] = (df["on thyroxine"] == "t").astype(int)
df["query on thyroxine"] = (df["query on thyroxine"] == "t").astype(int)
df["on antithyroid medication"] = (df["on antithyroid medication"]=="t").astype(int)
df["sick"] = (df["sick"]=="t").astype(int)
df["pregnant"] = (df["pregnant"]=="t").astype(int)
df["thyroid surgery"] = (df["thyroid surgery"]=="t").astype(int)
df["I131 treatment"] = (df['I131 treatment']=="t").astype(int)
df["query hypothyroid"] = (df["query hypothyroid"]=="t").astype(int)
df["lithium"] = (df["lithium"]=="t").astype(int)
df["goitre"] = (df["goitre"]=="t").astype(int)
df["tumor"] = (df["tumor"]=="t").astype(int)
df["hypopituitary"] = (df["hypopituitary"]=="t").astype(int)
df["query hyperthyroid"] = (df["query hyperthyroid"]=="t").astype(int)
df["psych"] = (df["psych"]=="t").astype(int)
df["TSH measured"] = (df["TSH measured"]=="t").astype(int)
df["T3 measured"] = (df["T3 measured"]=="t").astype(int)
df["TT4 measured"] = (df["TT4 measured"]=="t").astype(int)
df["T4U measured"] = (df["T4U measured"]=="t").astype(int)
df["FTI measured"] = (df["FTI measured"]=="t").astype(int)



In [535]:
from sklearn.impute import KNNImputer
np.random.seed(65)
knnimp = KNNImputer(n_neighbors=3)

cols = ['TSH','T3','TT4','T4U','FTI']
for i in cols:
    df[i] = knnimp.fit_transform(df[[i]])
df.isnull().sum()

age                          0
sex                          0
on thyroxine                 0
query on thyroxine           0
on antithyroid medication    0
sick                         0
pregnant                     0
thyroid surgery              0
I131 treatment               0
query hypothyroid            0
query hyperthyroid           0
lithium                      0
goitre                       0
tumor                        0
hypopituitary                0
psych                        0
TSH measured                 0
TSH                          0
T3 measured                  0
T3                           0
TT4 measured                 0
TT4                          0
T4U measured                 0
T4U                          0
FTI measured                 0
FTI                          0
binaryClass                  0
dtype: int64

In [536]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [537]:
print(len(train[train["binaryClass"]==1]))
print(len(train[train["binaryClass"]==0]))

2776
240


In [538]:
def scale_dataset(dataframe, oversample=False):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversample:
    ros = RandomOverSampler(sampling_strategy='minority')
    X, y = ros.fit_resample(X, y)

  data = np.hstack((X, np.reshape(y, (-1, 1))))

  return data, X, y

In [539]:
train, X_train, y_train = scale_dataset(train, oversample=True)
test, X_test, y_test = scale_dataset(test, oversample=False)

In [540]:
sum(y_train==1)


2776

In [541]:
sum(y_train==0)

2776

# KNN

In [542]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [543]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [544]:
y_pred = knn_model.predict(X_test)

In [545]:
lg_test_mcc = matthews_corrcoef(y_test, y_pred)
print('- MCC: %s' % lg_test_mcc)
print(classification_report(y_test, y_pred))

- MCC: 0.4087236340947416
              precision    recall  f1-score   support

           0       0.42      0.49      0.45        51
           1       0.96      0.95      0.96       704

    accuracy                           0.92       755
   macro avg       0.69      0.72      0.70       755
weighted avg       0.93      0.92      0.92       755



# NaiveBayes

In [546]:
from sklearn.naive_bayes import GaussianNB

In [547]:
nb_model = GaussianNB()
nb_model = nb_model.fit(X_train, y_train)

In [548]:
y_pred = nb_model.predict(X_test)
lg_test_mcc = matthews_corrcoef(y_test, y_pred)
print('- MCC: %s' % lg_test_mcc)
print(classification_report(y_test, y_pred,zero_division=0))

- MCC: 0.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.93      1.00      0.97       704

    accuracy                           0.93       755
   macro avg       0.47      0.50      0.48       755
weighted avg       0.87      0.93      0.90       755



# LOGISTIC REGRESSION

In [549]:
from sklearn.linear_model import LogisticRegression

In [550]:
lg_model = LogisticRegression()
lg_model = lg_model.fit(X_train, y_train)

In [551]:
y_pred = lg_model.predict(X_test)

lg_test_mcc = matthews_corrcoef(y_test, y_pred)
print('- MCC: %s' % lg_test_mcc)
print(classification_report(y_test, y_pred))

- MCC: 0.7764011988326792
              precision    recall  f1-score   support

           0       0.63      1.00      0.77        51
           1       1.00      0.96      0.98       704

    accuracy                           0.96       755
   macro avg       0.81      0.98      0.88       755
weighted avg       0.97      0.96      0.96       755



# SVM

In [552]:
from sklearn.svm import SVC

In [553]:
svm_model = SVC()
svm_model = svm_model.fit(X_train, y_train)

In [554]:
y_pred = svm_model.predict(X_test)
lg_test_mcc = matthews_corrcoef(y_test, y_pred)
print('- MCC: %s' % lg_test_mcc)
print(classification_report(y_test, y_pred))

- MCC: 0.8173252585745143
              precision    recall  f1-score   support

           0       0.70      0.98      0.82        51
           1       1.00      0.97      0.98       704

    accuracy                           0.97       755
   macro avg       0.85      0.98      0.90       755
weighted avg       0.98      0.97      0.97       755



# Decision Tree Classifier

In [555]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=3)
clf = dt.fit(X_train,y_train)
treepredict = clf.predict(X_test)
lg_test_mcc = matthews_corrcoef(y_test, y_pred)
print('- MCC: %s' % lg_test_mcc)
print(classification_report(y_test, treepredict))



- MCC: 0.8173252585745143
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        51
           1       1.00      0.99      1.00       704

    accuracy                           0.99       755
   macro avg       0.96      1.00      0.97       755
weighted avg       0.99      0.99      0.99       755



# RANDOM FOREST CLASSIFIER

In [556]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=2,n_estimators=200)
rclf = rf.fit(X_train,y_train)
rfpred = rclf.predict(X_test)
lg_test_mcc = matthews_corrcoef(y_test, y_pred)
print('- MCC: %s' % lg_test_mcc)
print(classification_report(y_test, rfpred))

- MCC: 0.8173252585745143
              precision    recall  f1-score   support

           0       0.65      1.00      0.78        51
           1       1.00      0.96      0.98       704

    accuracy                           0.96       755
   macro avg       0.82      0.98      0.88       755
weighted avg       0.98      0.96      0.97       755



# Neural Network

In [557]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=1, max_iter=1000)
mlp.fit(X_train, y_train)

# Make predictions
y_train_pred = mlp.predict(X_train)
y_test_pred = mlp.predict(X_test)

# Training set performance
mlp_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
mlp_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
mlp_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
mlp_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
mlp_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
mlp_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % mlp_train_accuracy)
print('- MCC: %s' % mlp_train_mcc)
print('- F1 score: %s' % mlp_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % mlp_test_accuracy)
print('- MCC: %s' % mlp_test_mcc)
print('- F1 score: %s' % mlp_test_f1)

Model performance for Training set
- Accuracy: 0.9902737752161384
- MCC: 0.9807331216138204
- F1 score: 0.9902728550335885
----------------------------------
Model performance for Test set
- Accuracy: 0.9682119205298013
- MCC: 0.8104432008587535
- F1 score: 0.9709637374836625


# Ensemble Stacking Classifier

In [558]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimator_list = [
    ('knn',knn_model),
    ('svm',svm_model),
    ('dt',dt),
    ('rf',rf),
    ('mlp',mlp),
     ]

# Build stack model
stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=LogisticRegression()
)

# Train stacked model
stack_model.fit(X_train, y_train)

# Make predictions
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)

# Training set model performance
stack_model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
stack_model_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
stack_model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
stack_model_train_precision = precision_score(y_train,y_train_pred)

# Test set model performance
stack_model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
stack_model_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
stack_model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
stack_model_test_precision = precision_score(y_test,y_test_pred)


print('Model performance for Training set')
print('- Accuracy: %s' % stack_model_train_accuracy)
print('- MCC: %s' % stack_model_train_mcc)
print('- F1 score: %s' % stack_model_train_f1)
print('- Precision score: %s' % stack_model_train_precision)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % stack_model_test_accuracy)
print('- MCC: %s' % stack_model_test_mcc)
print('- F1 score: %s' % stack_model_test_f1)
print('- Precision score: %s' % stack_model_test_precision)

Model performance for Training set
- Accuracy: 0.9994596541786743
- MCC: 0.9989198916740061
- F1 score: 0.9994596540209075
- Precision score: 1.0
----------------------------------
Model performance for Test set
- Accuracy: 0.990728476821192
- MCC: 0.9257776448832166
- F1 score: 0.9906858682991805
- Precision score: 0.9943262411347518
